In [12]:
import os
import sys
print "hey"
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline  
directory = "../data/"
files = os.listdir(directory)
print files

hey


OSError: [Errno 2] No such file or directory: '../data/'

In [ ]:
# plot x-axis = N, y axis = metrics
def paint(fn, plot_datas, mode):
    
    if mode == 1: # plot N - final result
        x_label = plot_datas.keys() #N 
        print "this is tested N: " + str(x_label)
        range_limit = 9999999999
        for i in x_label:
            range_limit = min(range_limit, len(plot_datas[i][0])) # plot_datas[i] => data, data[0] => len(col) => lines
        print "range limit, lines of data for %d" %  range_limit # TODO data is picked from the last aggregation cell(end of program)
        plot_data = []
        
         # data_index = range_limit - 1
        for typeindex in [1,2,3, 4, 5]: # min, maj, cputime, runtime, utilization, 1235 target
            plot_data.append([plot_datas[i][typeindex][-1] for i in x_label ])
        print plot_data
        title1 = 'N_page_fault'
        title2 = "N_cpu_time_and_run_time"
        title3 = "N_cpu_time_utilization_rate"
        
        plt.title(title1)
        plt.plot( x_label, plot_data[0], "r+", x_label, plot_data[1], "ro")  # pg_flt - N 
        plt.xlabel("N")
        plt.ylabel("cumulative page fault, (times)")
        fig1=plt.gcf()
        fig1.savefig(title1 + '.png',  bbox_inches='tight')
        plt.show()
        plt.close()
        
        plt.title(title2)
        plt.plot( x_label, plot_data[2], 'g+', x_label, plot_data[3], 'go') # utime and stime
        plt.xlabel("N")
        plt.ylabel("cumulative cpu_time and run_time of program(jiffiies)")
        fig2=plt.gcf()
        fig2.savefig(title2 + '.png',  bbox_inches='tight')
        plt.show()
        plt.close()
        
        plt.title(title3)
        plt.plot( x_label, plot_data[4], 'b+')
        plt.xlabel("N")
        plt.ylabel("utilization rate(cpu_time / runtime)")
        fig3=plt.gcf()
        fig3.savefig(title3 + '.png', bbox_inches='tight')
        plt.show()
        plt.close()
        
        # Create plots with pre-defined labels.
#     fig, ax = plt.subplots()
#     ax.plot(a, c, 'k--', label='Model length')
#     ax.plot(a, d, 'k:', label='Data length')
#     ax.plot(a, c + d, 'k', label='Total message length')

#     legend = ax.legend(loc='upper center', shadow=True, fontsize='x-large')

#     # Put a nicer background color on the legend.
#     legend.get_frame().set_facecolor('#00FFCC')


    else:
        title1 = fn + "- time - page fault"
        title2 = fn + "- time - cpu time"
        rang = len(plot_datas[0])
        print 'length of data: %d' % rang
        x_label = plot_datas[4]
        # typeindex in [1,2,3, 4, 5]: # min, maj, cputime, runtime, utilization, 1235 target
        plt.title(title1)
        plt.plot( x_label, plot_datas[0], "r+", x_label, plot_datas[1], "ro")  # pg_flt
        plt.xlabel("time(s)")
        plt.ylabel("data-page fault, minor(+) and major(o)")
        fig=plt.gcf()
        plt.show()
        fig.savefig('case12_' + fn + '.png', bbox_inches='tight')
        plt.close()
        
#         plt.title(title2)
#         plt.plot( x_label, plot_datas[2], 'g+', x_label, plot_datas[3], 'go')
#         plt.xlabel("times(s)")
#         plt.ylabel("cpu time")
#         plt.show()
        

In [ ]:
# for N iterations: 

def readData(data, directory, fn):
    fp = open(directory + fn, "r+")
    data.append([]) # time/ jiffiy
    data.append([]) # minor fault
    data.append([]) # major fault
    data.append([]) # utime+stime
    data.append([]) # current runtime/ jiffiy
    data.append([]) # utilization = (u+s) / runtime
    # print data
    
    flag=True
    start_time = sys.maxsize
    for line in fp:
        data_line = line.strip().split(" ")
        # datas = line
        if len(data_line) != 4 or data_line[0] == 'read' or data_line[0] == 'file': # eliminate the error state
            print data_line
            continue
        if (flag):
            start_time = int(data_line[0])
            flag = False # init for the start time
            data[0].append(int(data_line[0])) 
            data[4].append(int([0] - start_time))
            data[1].append(int(data_line[1]))
            data[2].append(int(data_line[2]))
            data[3].append(int(data_line[3]))
            data[5].append(float(data[3][-1]) / data[4][-1])
        # cumulative for fault, cputime |  run time: time - start of row[0] | utilization: aggregate cputime/ runtime
        else:
            data[0].append(int(data_line[0])) 
            data[4].append(int([0] - start_time))
            data[1].append(int(data_line[1]) + data[1][-1])
            data[2].append(int(data_line[2]) + data[2][-1])
            data[3].append(int(data_line[3]) + data[3][-1])
            data[5].append(float(data[3][-1]) / data[4][-1])  
    assert(len(data[0]) == len(data[1])and len(data[1]) == len(data[3]) && len(data[3]) == len(data[5]))

        # the size of int in python is 32bit as large as unsigned long in C
    print "data read finished for file %s, %d lines of data read in " % (fn, len(data[4]))
    fp.close()


fnames = []
datas_exp3 = {} # N related
fn_exp3 = {}
datas_exp12 = {} # experiments
fn_exp12 = {}

for f in files:
    index = f.find("N")
    if index != -1:
        s = f[index:]
        s = s.split(".")[0][1:]
        # print "N = " + s # N
        N = int(s)
        data = []
        datas_exp3[N] = data
        fn_exp3[N] = f
        readData(data, directory, f)
        print len(datas_exp3[N][0]) # test
    else:  # experiment 1 & 2
        N = int(f.split(".")[0][-1])
        print 'exp12:  ', N
        data = []
        datas_exp12[N] = data
        fn_exp12[N] = f
        readData(data, directory, f)
    
def paint_tool(exp, idx, mode):
    if exp == 3:
        if mode == 1: # print N => 
            paint(fn_exp3, datas_exp3, mode)
        else:
            paint(fn_exp3[idx], datas_exp3[idx], mode)
    else: # exp == 1 print single  time related data chart
        paint(fn_exp12[idx], datas_exp12[idx], 0)

# experiment 3, N = 5
# paint_tool(3, 5, 0)

# experiment 1,2 
print fn_exp12, 'fn_exp12'
paint_tool(1, 1, 0)
paint_tool(1, 2, 0) # run the second experiment

# experiment 3, N = 3 5 111，
paint_tool(3, 0, 1)

print "finished" 